# UC San Diego: Neural Data Science
## Estimating Song Tempo from EEG signals

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [x] YES - make available
* [  ] NO - keep private

# Names

- Sidney Ma
- Dibyesh Sahoo
- Ryan Ho
- Ishaan Chadha

# Overview

We analyzed EEG signals from a group of people listening to naturalistic songs in order to predict the tempo of the song. First, we averaged the EEG signals of participants listening to a single song and applied PCA to extract meaningful data. This approach was based on a previous study that found tempo is related to spikes in the PCA signal. We then applied this method to a new dataset and used smoothing algorithms to enhance the accuracy of tempo calculations by identifying the peaks in the signal.

<a id='research_question'></a>
# Research Question

Using only EEG data of participants listening to music, is it possible to accurately predict the tempo of the songs they were listening to?

<a id='background'></a>

# Background & Prior Work

Tempo describes the speed or pace of a song. We as humans are very good at "feeling" the tempo of a song, even when we're not aware of it. Even though audio waveforms of songs are difficult to interpret on their own, it is very easy to hear structure and patterns in them. This alone should suggest that there is some higher-level processing in the brain that can identify tempo and help us "feel" the beat (Quinn and Watt 2006). Thus, we should expect to see some indication of this in electrocortical recordings when people listen to music.

This is what Kaneshiro et al. found in their 2017 study – after only basic processing of EEG data, they revealed strong oscillations at frequencies related to the tempo and beat structure of a listened song. In 2020, they reinforced this in another study, where this effect occured even in reversed or reshuffled songs, so long as the tempo was the same.

However, while these studies show a clear relationship between EEG data and tempo, there is not yet any good way to quantify this relationship, nor is there a way to predict the tempo using only the EEG data. Our goal in this project is to develop a pipeline that can input pre-processed EEG data and output an estimated tempo, without having any prior knowledge of the song. This could expand the field of music information retrieval (MIR) and quantify the ability to retrieve cortical information. Additionally, prediction errors could provide insight as to what makes some songs "easy" or "hard" to follow along to. In general, it is a tool that will help us better understand how auditory information is processed by humans, and how raw signals can be decoded into meaningful structures.

References:
- https://pubmed.ncbi.nlm.nih.gov/16583770/
- https://ccrma.stanford.edu/~blairbo/assets/pdf/losorelli2017ISMIR.pdf
- https://www.sciencedirect.com/science/article/pii/S105381192030046X?via%3Dihub

### About tempo

Tempo is typically defined in beats per minute (bpm), but it can also be defined in Hz (beats per second), a conversion factor of 60. When listeners feel a tempo, they also feel tempo-related frequencies, which are typically in a power series of 2. Specifically, given tempo $T$, tempo-related frequencies are $T*2^N $ where $N$ is any integer. As Kaneshiro et al. found, oscillations spike at tempo-related frequencies, not just the tempo frequency. As a result, it is very difficult to determine which tempo-related frequency is the actual tempo.

For example, if a song has a tempo of 160 bpm (2.7 Hz), then 80 bpm (1.3 Hz) is a tempo-related frequency, and strong spikes would likely show up at both frequencies. From these spikes alone, it would be basically impossible to know whether the song was 160 or 80 bpm (even humans might disagree about the tempo!). For this reason, we are choosing to normalize the tempo estimates to be between 60 and 120 bpm (1 and 2 Hz). Any tempo has exactly one tempo-related frequency in this range. For the purpose of our project, if a song has a tempo outside of this range, we will simply choose the tempo-related frequency that is inside this range, and interpret that to be the true tempo.

# Hypothesis


Given the relationship between oscillation spikes and tempo, we expect that it is possible to estimate the tempo accurately. However, we will not be able to account for octave differences (i.e. our algorithm will not distinguish tempo $T$ from tempo $T*2^N$), and our tool will only provide estimates between 60 and 120 bpm (1 and 2 Hz).

# Dataset(s)

- Dataset Name: Naturalistic Music EEG Dataset - Tempo (NMED-T)
- Link to the dataset: https://purl.stanford.edu/jn859kj8079
- Number of observations: 10

This dataset contains preprocessed EEG data collected from participants while they listened to 10 different songs. Each observation is a 125 x 34795 x 20 matrix, representing the electrodes, timepoints, and participants, respectively. It uses a sample rate of 125, meaning each recording was about 4 minutes 28 seconds long.

- Dataset Name: Naturalistic Music EEG Dataset - Hindi (NMED-H)
- Link to the dataset: https://purl.stanford.edu/sd922db3535
- Number of observations: 8

This dataset contains preprocessed EEG data collected from participants while they listened to 4 different Hindi songs; there are 8 observations because there were two listening sessions (denoted *a* and *b*) for each song. Each observation is a 125 x 33271 x 12 matrix, representing the electrodes, timepoints, and participants, respectively. It also uses a sample rate of 125, meaning each recording was about 4 minutes 26 seconds long.

The preprocessing is the same for both datasets, and is described in the corresponding papers.

# Data Wrangling

In [1]:
import numpy as np, pandas as pd
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt

import scipy.io
from scipy.fft import fft, fftfreq
from scipy.ndimage import gaussian_filter1d

from tqdm import tqdm
from helpers import *

We downloaded the EEG data directly from the dataset web pages into a personal computer. In total, this amounted to 10 GB. They are matricies are stored as .mat files, with the names listed below:

In [2]:
# Filenames for NMED-T
data_t_filenames = ['song21_Imputed.mat',
 'song22_Imputed.mat',
 'song23_Imputed.mat',
 'song24_Imputed.mat',
 'song25_Imputed.mat',
 'song26_Imputed.mat',
 'song27_Imputed.mat',
 'song28_Imputed.mat',
 'song29_Imputed.mat',
 'song30_Imputed.mat']

# Filenames for NMED-H
data_h_filenames = ['song21_a_Imputed.mat',
 'song21_b_Imputed.mat',
 'song22_a_Imputed.mat',
 'song22_b_Imputed.mat',
 'song23_a_Imputed.mat',
 'song23_b_Imputed.mat',
 'song24_a_Imputed.mat',
 'song24_b_Imputed.mat']

## Initial Processing

Since none of us own MATLAB, we needed to pull the files in as ndarrays in order to work with them in python. However, if we were to pull in all 14 matricies at once, this would not only use up a lot of memory, but would also take way too long to process. Instead, we pulled in each matrix and ran some initial processing (including dimensionality reduction) before moving onto the next.

Our process replicates the one described in Kaneshiro et al. (2017), which involves these three steps:
1. Compute the mean signal across participants
2. Compute the first pricipal component (PC1) of the mean signal
3. Run a Fast Fourier Transform (FFT) on the PC1


In [3]:
sr = 125 # Sample rate is 125 for both datasets
def collect_pc1_data(eeg, output_name):
    # Mean eeg signal across participants
    mean_eeg = np.mean(eeg, axis=2)
    mean_eeg_flat = mean_eeg.reshape(125, -1)

    # PC1
    U, S, Vt = np.linalg.svd(mean_eeg_flat, full_matrices=False)
    PC1 = U[:, 0]
    PC1_time_series = PC1.dot(mean_eeg_flat)

    # FFT
    N = PC1_time_series.shape[0]
    yf = fft(PC1_time_series)
    xf = fftfreq(N, 1/sr)
    freq, power = xf[:N//2], np.abs(yf[:N//2])
    pc1_data = np.vstack((freq, power))
    
    np.save(output_name, pc1_data)

We saved each reduced dataset into the folders pc1-T (for NMED-T) and pc1-H (for NMED-H). Note that for NMED-H, it vertically stacks the a and b matricies, effectively doubling the number of participants.

In [4]:
for i in tqdm(range(10)):
    source_name = f"data-T/{data_t_filenames[i]}"
    column_name = f"data{i + 21}"
    
    mat = scipy.io.loadmat(source_name)
    eeg = mat[column_name]
    
    collect_pc1_data(eeg, f"pc1-T/t{i+1}")

for i in tqdm(range(4)):
    source_a_name = f"data-H/song{i + 21}_a_Imputed.mat"
    source_b_name = f"data-H/song{i + 21}_b_Imputed.mat"
    column_a_name = f"data{i + 21}_a"
    column_b_name = f"data{i + 21}_b"
    
    mat_a = scipy.io.loadmat(source_a_name)
    eeg_a = mat_a[column_a_name]
    
    mat_b = scipy.io.loadmat(source_b_name)
    eeg_b = mat_b[column_b_name]
    
    eeg = np.vstack((eeg_a, eeg_b)) # treat A and B entries like different participants
    
    collect_pc1_data(eeg, f"pc1-H/h{i+1}")

# Data Visualization

## Initial processing explained

First, to illustrate the above process:

![](images/p5.png)

Our process involved computing the mean signal, the PC1, and the FFT of each EEG matrix. While we could have simply computed the FFT of the mean signal (top right), this does not display any prominent spikes, which is necessary for predicting the tempo.

## Observing tempo lineups for NMED-H

Next, we replicated the plot in Kaneshiro et al. (2017), using the four Hindi songs instead:

![](images/p1.png)

The above shows the FFT signals for the four Hindi songs. Using the tempos provided in Kaneshiro et al. (2020), we plotted vertical lines at tempo-related frequencies, that is $T*2^N$ for $N$ in range $[-3, 3]$. As expected, the peaks in the FFT signals did seem to line up with tempo-related frequencies.

# Data Analysis
While it is clear that the FFT signals are related to the tempo, it is not immediately clear how they can be used to predict the tempo. We devised three high-level steps to accomplish this:
1. Log the frequency spectrum so that the peaks are more spread out
2. Find the frequency positions of the "prominent" peaks
3. Use these frequency positions to estimate the tempo

### Step 1: Log frequency

Both the peaks and the tempo-related frequencies are squished together at low frequencies, so it is necessary to view the frequency spectrum on a log scale in order to identify them.

In [6]:
def get_log_freq(freq, power):
    freq = np.where(freq == 0, freq + 0.00001, freq) # avoid log(0) issues
    freq = np.log2(freq)
    mask = (freq >= -3) & (freq <= 5) # only include positions between 1/8 Hz and 32 Hz
    freq = freq[mask]
    power = power[mask]
    
    return freq, power

![](images/p2.png)

### Step 2: Find peaks
The goal here is to locate the frequency positions of "prominent" peaks. There are a variety of challenges within this step. Firstly, the FFT signals are basically entirely composed of peaks, so it is difficult to identify the "prominent" ones from the "normal" ones. Secondly, the FFTs all follow a non-linear trend, which means that the peaks are relative, not absolute. Thirdly, because the frequency axis is logscaled, peaks closer to 0 Hz will be much wider than those at higher frequencies.

To accomplish our goal, we implemented a variety of simple statistical methods to emphasize the "prominent" peaks and dampen the "normal" ones:
- Moving maximum: a windowing function that returns the maximum value of the window. This effectively "connects the dots," tracing out large peaks and ignores smaller ones.
- Exclusive moving average: a windowing function that returns the average value of the window, excluding the current point. This accounts for the non-linear trend, setting the peaks onto an "even playing field." By excluding the current point, it emphasizes any "sudden" differences that appear at peaks.
- 1D Gaussian filter: a smoothing function that helps us identify peaks by finding the local maxima.

The pipeline is described in code and illustrated below:

In [7]:
def get_peaks(freq, power):
    A = mov_max(power, 10)
    B = mov_avg_exc(A, 100)
    C = heavyside(A - B)
    D = mov_max(C, 20)**2
    curve = gaussian_filter1d(D, 5)
    
    # Find peaks
    peaks = np.where((curve[1:-1] > curve[:-2]) & (curve[1:-1] > curve[2:]))[0] + 1
    
    return curve, peaks

![](images/p3.png)

We were only able to decide on the parameters for the function through experimenting with different values.

### Step 3: Estimating tempo
Now that the peaks are identified, the next challenge is to estimate the tempo. There are three sub-tasks involved in this step:
1. Finding the frequencies of the largest peaks
2. Normalizing the frequencies
3. Deciding which frequency is most likely to be the tempo

Finding the frequencies of the largest peaks is somewhat difficult, but only involves dataframe sorting. After some experimenting, we decided that the top 6 largest peaks were most representative of the tempo – anything lower was likely to be random.

Normalizing the frequencies means selecting the tempo-related frequency between 1 and 2 Hz. To do this, we must multiply or divide each frequency by 2 until it is between 1 and 2 Hz. In log space, this is simply finding the decimal value of the log frequency. 

Deciding on the correct frequency is difficult – of the 6 selected frequencies, typically only 3 or 4 are actually close the the correct tempo. Central tendency measures like mean or median don't return good results, as they are overly influenced by bad estimates. After some experimentation, we decided on using the medioid as the best estimate. This way, good estimates would be much more likely than bad estimates to be the final guess.

Importantly, due to the cyclical nature of tempo, the similarity of two tempos should be quantified as the circular distance between them, not the arithmetic difference.

In [8]:
def estimate_tempo(freq, curve, peaks):
    # Find frequencies of largest peaks
    df = pd.DataFrame((freq[peaks], curve[peaks])).T
    df.columns = "freq", "power"
    df_sort = df.sort_values(by="power", ascending=False).head(6)
    peaks_sorted = df_sort["freq"]
    
    # Normalize the frequencies
    peaks_normalized = (peaks_sorted % 1).to_list()
    
    # Decide which frequency is most likely
    similarity_matrix = [[circ_diff(a, b) for b in peaks_normalized] for a in peaks_normalized]
    similarity_matrix = np.array(similarity_matrix)
    most_accurate_idx = np.argmin(similarity_matrix.sum(axis=0))
    medioid = peaks_normalized[most_accurate_idx]
    
    return medioid

### Putting it all together

In [9]:
# Input the PC1 data, output the tempo guess
def estimate_tempo_from_pca(freq, power):
    log_freq, power_mask = get_log_freq(freq, power)
    curve, peaks = get_peaks(log_freq, power_mask)
    return estimate_tempo(log_freq, curve, peaks)

In [10]:
# Estimate tempos for NMED-T
estimated_tempos_t = []
for i in range(10):
    filename = f"pc1-T/t{i+1}.npy"
    freq, power = np.load(filename)
    estimated_tempos_t.append(estimate_tempo_from_pca(freq, power))
    
# Estimate tempos for NMED-H
estimated_tempos_h = []
for i in range(4):
    filename = f"pc1-H/h{i+1}.npy"
    freq, power = np.load(filename)
    estimated_tempos_h.append(estimate_tempo_from_pca(freq, power))

# Results
Using the true tempo values provided in both Kaneshiro et al. studies, we evaluated the predicted tempos. The results are displayed in the following table:

In [15]:
tempos_t = [0.9328, 1.1574, 1.2376, 1.3736, 1.5244, 1.6026, 1.8116, 2.0000, 2.1368, 2.5000]
tempos_h = [2.60, 1.56, 1.50, 1.43]

df = pd.DataFrame(tempos_t + tempos_h, columns = ["hz"]) # true tempos
df["bpm"] = df["hz"] * 60 # in bpm
df["log_hz"] = df["hz"].apply(np.log2) # logged
df["hz_norm"] = df["log_hz"] % 1 # normalized between 1 and 2 hz

df["hz_estimated"] = estimated_tempos_t + estimated_tempos_h # estimated tempos
df["error"] = df.apply( # error using circular difference
    lambda row: circ_diff(row["hz_norm"], row["hz_estimated"]), axis=1)

# Display results
print("Mean error:", df["error"].median())
df

Mean error: 0.0057879864287789795


,hz,bpm,log_hz,hz_norm,hz_estimated,error
0,0.9328,55.968,-0.100360,0.899640,0.873545,0.026094
1,1.1574,69.444,0.210888,0.210888,0.220950,0.010063
2,1.2376,74.256,0.307545,0.307545,0.306400,0.001145
3,1.3736,82.416,0.457962,0.457962,0.442340,0.015622
4,1.5244,91.464,0.608242,0.608242,0.616677,0.008435
5,1.6026,96.156,0.680414,0.680414,0.677031,0.003384
6,1.8116,108.696,0.857264,0.857264,0.861163,0.003898
7,2.0000,120.000,1.000000,0.000000,0.000104,0.000104
8,2.1368,128.208,1.095452,0.095452,0.101028,0.005576
9,2.5000,150.000,1.321928,0.321928,0.322083,0.000155


![](images/p4.png)

As evident from the table and plot, our estimated tempos were very accurate. Only one song had an error ratio larger than 3%, and the median error ratio was 0.58%.

The outlier song (Song 12) is "Haule Haule" from "Rab Ne Bana Di Jodi" (2008). Looking at the following graph, it is understandable why the algorithm failed on this one:

![](images/p6.png)

It seems that the tempo is lined up with two of the peaks, but misses the ones in the middle.

# Conclusion & Discussion

Although tempo can be hard to interpret from a song's waveform, it is very easy, even automatic, for humans to find. This is only further evidenced by our results, which indicate that a song's tempo is not only linked to the corresponding EEG data, but can even be predicted by it.

The main limitation our project faces is that it cannot predict any tempos outside of its specified range (60 - 120 bpm). While this would be difficult to do, there should still be certain artifacts in the FFT signal that could indicate whether or not the tempo is outside of this range. Our tool will also not work with uneven time signatures, as we only consider temporal structure with powers of two. Apart from that, we have not tested the external validity of our project, which is important considering that both datasets come from the same researchers, and likely the same equipment.

The failure to identify the correct tempo for Song 12 shows that many of the peaks are not at tempo-related frequencies, or at least not in the tempo-related power series. Future work could investigate what frequencies these peaks lie on and how, if at all, they relate to the main tempo.

Overall, we are happy with the results, and hope to see similar work in the future.